In [18]:
import keyboard
import os
import tempfile

import numpy as np
import openai
import sounddevice as sd
import soundfile as sf
import tweepy

# from elevenlabs import generate, play, set_api_key
# from langchain.agents import initialize_agent, load_tools
# from langchain.agents.agent_toolkits import ZapierToolkit
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
# from langchain.tools import BaseTool
# from langchain.utilities.zapier import ZapierNLAWrapper

from elevenlabs.client import ElevenLabs
from elevenlabs import Voice, VoiceSettings, play

from dotenv import load_dotenv
import os

from openai import OpenAI
openai_client = OpenAI()

load_dotenv()

# set_api_key(os.getenv("ELEVEN_API_KEY"))
# openai.api_key = os.get("OPENAI_API_KEY")

# Set recording parameters
duration = 5  # duration of each recording in seconds
fs = 44100  # sample rate
channels = 1  # number of channels


def record_audio(duration, fs, channels):
    print("Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=channels)
    sd.wait()
    print("Finished recording.")
    return recording


def transcribe_audio(recording, fs):
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
        sf.write(temp_audio.name, recording, fs)
        temp_audio.close()
        with open(temp_audio.name, "rb") as audio_file:
            # transcript = openai.Audio.transcribe("whisper-1", audio_file)
            transcription = openai_client.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file, 
                response_format="text"
            )
        os.remove(temp_audio.name)
    return transcription


client = ElevenLabs(
    api_key=os.getenv("XI_API_KEY")  # Load API key from environment variable
)

def play_generated_audio(text):

    # Generate the audio response using Eleven Labs API
    # audio = client.generate(
    #     text=text,
    #     voice="Aria",  # You can replace "Aria" with any other voice ID or name
    #     model="eleven_multilingual_v2",  # Specify the model version to use
    #     stream=True,
    #     voice_settings={
    #         "stability": 0.8,  # Adjust stability (0.0 - 1.0)
    #         "similarity_boost": 0.75  # Adjust the likeness to the selected voice
    #     }
    # )
    # play(audio)
    
    response = openai_client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=text
    )

    if hasattr(response, 'content') and response.content:
        # Use your play function to play the audio instantly
        play(response.content)
    elif hasattr(response, 'url'):
        # If the API provides a URL, use your play function to play the audio from URL
        audio_url = response.url
        play(audio_url)
    else:
        print("No valid audio content found in the response.")    


In [19]:
# openai_client.audio.speech.create(
#     model="tts-1",
#     voice="alloy",
#     input="Hello, how are you?"
# )



In [20]:
llm = ChatOpenAI(name="gpt-4o", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history")

# zapier = ZapierNLAWrapper(zapier_nla_api_key="<ZAPIER_NLA_API_KEY>")
# toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)

# tools = [TweeterPostTool()] + toolkit.get_tools() + load_tools(["human"])

# tools = toolkit.get_tools() + load_tools(["human"])

# agent = initialize_agent(tools, llm, memory=memory, agent="conversational-react-description", verbose=True)

while True:
    print("Press spacebar to start recording.")
    keyboard.wait("space")  # wait for spacebar to be pressed
    recorded_audio = record_audio(duration, fs, channels)
    message = transcribe_audio(recorded_audio, fs)
    print(f"You: {message}")
    # assistant_message = agent.run(message)
    assistant_message = llm.invoke(message).content
    
    play_generated_audio(assistant_message)

C:\Users\pablo\AppData\Local\Temp\ipykernel_2440\4113104061.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


Press spacebar to start recording.
Recording...
Finished recording.
You: Hej, hur mår du?

Press spacebar to start recording.
Recording...
Finished recording.
You: Üzgünüm adamım.

Press spacebar to start recording.


KeyboardInterrupt: 

: 